In [183]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances

In [184]:
df_cuisine = pd.read_csv('./inputs/chefmozcuisine.csv')
df_cuisine.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


In [185]:
df_parking = pd.read_csv('./inputs/chefmozparking.csv')
df_parking.head()
df_parking['parking_lot'].unique()

array(['public', 'none', 'yes', 'valet parking', 'fee', 'street',
       'validated parking'], dtype=object)

In [186]:
df_geoplaces = pd.read_csv('./inputs/geoplaces2.csv')
df_geoplaces = df_geoplaces[['placeID', 'name', 'alcohol', 'smoking_area', 'dress_code', 'accessibility', 'price', 'Rambience']]
df_geoplaces.head()
print(df_geoplaces['alcohol'].unique())
print(df_geoplaces['smoking_area'].unique())
print(df_geoplaces['dress_code'].unique())
print(df_geoplaces['accessibility'].unique())
print(df_geoplaces['price'].unique())
print(df_geoplaces['Rambience'].unique())

['No_Alcohol_Served' 'Wine-Beer' 'Full_Bar']
['none' 'only at bar' 'permitted' 'section' 'not permitted']
['informal' 'casual' 'formal']
['no_accessibility' 'completely' 'partially']
['medium' 'low' 'high']
['familiar' 'quiet']


In [187]:
df_ratings = pd.read_csv('./inputs/rating_final.csv')
df_ratings['overall_rating'] = (df_ratings['rating'] + df_ratings['food_rating'] + df_ratings['service_rating'])
df_ratings.head()

,userID,placeID,rating,food_rating,service_rating,overall_rating
0,U1077,135085,2,2,2,6
1,U1077,135038,2,2,1,5
2,U1077,132825,2,2,2,6
3,U1077,135060,1,2,2,5
4,U1068,135104,1,1,2,4


In [188]:
df = pd.merge(df_ratings, df_geoplaces, on='placeID')
df = pd.merge(df, df_cuisine, on='placeID')
df = pd.merge(df, df_parking, on='placeID')
df.head()

,userID,placeID,rating,food_rating,service_rating,overall_rating,name,alcohol,smoking_area,dress_code,accessibility,price,Rambience,Rcuisine,parking_lot
0,U1077,135085,2,2,2,6,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,Fast_Food,public
1,U1108,135085,1,2,1,4,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,Fast_Food,public
2,U1081,135085,1,2,1,4,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,Fast_Food,public
3,U1056,135085,2,2,2,6,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,Fast_Food,public
4,U1134,135085,2,1,2,5,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,Fast_Food,public


In [189]:
# Parking Encoding
df['parking_lot'] = df['parking_lot'].map({'public':2, 'none':0, 'yes':1, 'valet parking': 1, 'fee':1, 'street':1, 'validated parking':1})
df

,userID,placeID,rating,food_rating,service_rating,overall_rating,name,alcohol,smoking_area,dress_code,accessibility,price,Rambience,Rcuisine,parking_lot
0,U1077,135085,2,2,2,6,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,Fast_Food,2
1,U1108,135085,1,2,1,4,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,Fast_Food,2
2,U1081,135085,1,2,1,4,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,Fast_Food,2
3,U1056,135085,2,2,2,6,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,Fast_Food,2
4,U1134,135085,2,1,2,5,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,Fast_Food,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,U1061,132958,2,2,2,6,tacos los volcanes,No_Alcohol_Served,none,informal,completely,low,quiet,American,0
1039,U1025,132958,1,0,0,1,tacos los volcanes,No_Alcohol_Served,none,informal,completely,low,quiet,American,0
1040,U1097,132958,2,1,1,4,tacos los volcanes,No_Alcohol_Served,none,informal,completely,low,quiet,American,0
1041,U1096,132958,1,2,2,5,tacos los volcanes,No_Alcohol_Served,none,informal,completely,low,quiet,American,0


In [190]:
# Geo Encoding Based on Categories
df['alcohol'] = df['alcohol'].map({'No_Alcohol_Served':0, 'Wine-Beer':1, 'Full_Bar':2})
df['smoking_area'] = df['smoking_area'].map({'none':0, 'only at bar':1, 'permitted':1, 'section':0, 'not permitted':0})
df['dress_code'] = df['dress_code'].map({'informal':0, 'casual':1, 'formal':2})
df['accessibility'] = df['accessibility'].map({'no_accessibility':0, 'completely':1, 'partially':2})
df['price'] = df['price'].map({'medium':0, 'low':1, 'high':2})
df['Rambience'] = df['Rambience'].map({'familiar':0, 'quiet':1})
df

,userID,placeID,rating,food_rating,service_rating,overall_rating,name,alcohol,smoking_area,dress_code,accessibility,price,Rambience,Rcuisine,parking_lot
0,U1077,135085,2,2,2,6,Tortas Locas Hipocampo,0,0,0,0,0,0,Fast_Food,2
1,U1108,135085,1,2,1,4,Tortas Locas Hipocampo,0,0,0,0,0,0,Fast_Food,2
2,U1081,135085,1,2,1,4,Tortas Locas Hipocampo,0,0,0,0,0,0,Fast_Food,2
3,U1056,135085,2,2,2,6,Tortas Locas Hipocampo,0,0,0,0,0,0,Fast_Food,2
4,U1134,135085,2,1,2,5,Tortas Locas Hipocampo,0,0,0,0,0,0,Fast_Food,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,U1061,132958,2,2,2,6,tacos los volcanes,0,0,0,1,1,1,American,0
1039,U1025,132958,1,0,0,1,tacos los volcanes,0,0,0,1,1,1,American,0
1040,U1097,132958,2,1,1,4,tacos los volcanes,0,0,0,1,1,1,American,0
1041,U1096,132958,1,2,2,5,tacos los volcanes,0,0,0,1,1,1,American,0


In [191]:
# Rcuisine encoding
df['Rcuisine'] = df['Rcuisine'].map(
    {
        'Spanish':0,
        'Italian':1,
        'Latin_American':2,
        'Mexican': 3,
        'Fast_Food':4,
        'Burgers':5,
        'Dessert-Ice_Cream':6,
        'Hot_Dogs':7,
        'Steaks':8,
        'Asian':9,
        'International':10,
        'Mongolian':11,
        'Vegetarian':12,
        'Brazilian':13,
        'Cafe-Coffee_Shop':14,
        'Cafeteria':15,
        'Contemporary':16,
        'Deli-Sandwiches':17,
        'Diner':18,
        'Japanese':19,
        'Sushi':20,
        'Seafood':21,
        'Chinese':22,
        'Bar':23,
        'Bar_Pub_Brewery':24,
        'Pizzeria':25,
        'Mediterranean':26,
        'American':27,
        'Family':28,
        'Caribbean':29,
        'African':30,
        'Breakfast-Brunch':31,
        'Regional':32,
        'Afghan':33,
        'Bakery':34,
        'Game':35,
        'Armenian':36,
        'Vietnamese':37,
        'Korean':38,
        'Thai':39,
        'Barbecue':40,
        'Polish':41,
        'Dutch-Belgian':42,
        'French':43,
        'German':44,
        'Southwestern':45,
        'Persian':46,
        'Ethiopian':47,
        'Juice':48,
        'Soup':49,
        'Continental-European':50,
        'Greek':51,
        'Southern':52,
        'Eastern_European':53,
        'California':54,
        'Bagels':55,
        'Turkish':56,
        'Organic-Healthy':57,
        'Fine_Dining':58
    }
)
df.head()

,userID,placeID,rating,food_rating,service_rating,overall_rating,name,alcohol,smoking_area,dress_code,accessibility,price,Rambience,Rcuisine,parking_lot
0,U1077,135085,2,2,2,6,Tortas Locas Hipocampo,0,0,0,0,0,0,4,2
1,U1108,135085,1,2,1,4,Tortas Locas Hipocampo,0,0,0,0,0,0,4,2
2,U1081,135085,1,2,1,4,Tortas Locas Hipocampo,0,0,0,0,0,0,4,2
3,U1056,135085,2,2,2,6,Tortas Locas Hipocampo,0,0,0,0,0,0,4,2
4,U1134,135085,2,1,2,5,Tortas Locas Hipocampo,0,0,0,0,0,0,4,2


In [192]:
# generate encoded string
df['encoded_values'] = (df['overall_rating'].map(str)+df['alcohol'].map(str)).map(int)

df['encoded_values'] = (
        df['overall_rating'].map(str) + 
        '.' +
        df['alcohol'].map(str) + 
        df['smoking_area'].map(str) + 
        df['dress_code'].map(str) + 
        df['accessibility'].map(str) + 
        df['price'].map(str) + 
        df['Rambience'].map(str) + 
        df['Rcuisine'].map(str) + 
        df['parking_lot'].map(str)
    ).astype(np.float64)

df.head()

,userID,placeID,rating,food_rating,service_rating,overall_rating,name,alcohol,smoking_area,dress_code,accessibility,price,Rambience,Rcuisine,parking_lot,encoded_values
0,U1077,135085,2,2,2,6,Tortas Locas Hipocampo,0,0,0,0,0,0,4,2,6.0
1,U1108,135085,1,2,1,4,Tortas Locas Hipocampo,0,0,0,0,0,0,4,2,4.0
2,U1081,135085,1,2,1,4,Tortas Locas Hipocampo,0,0,0,0,0,0,4,2,4.0
3,U1056,135085,2,2,2,6,Tortas Locas Hipocampo,0,0,0,0,0,0,4,2,6.0
4,U1134,135085,2,1,2,5,Tortas Locas Hipocampo,0,0,0,0,0,0,4,2,5.0


In [194]:
train_data, test_data = train_test_split(df, test_size=0.25)
restmatrix_train_data = train_data.pivot_table(index='userID',columns='name',values='encoded_values')
restmatrix_test_data = test_data.pivot_table(index='userID',columns='name',values='encoded_values')

restmatrix = df.pivot_table(index='userID',columns='name',values='encoded_values')

train_data_RMSE = train_data.pivot_table(index='userID',columns='name',values='encoded_values')
test_data_RMSE = test_data.pivot_table(index='userID',columns='name',values='encoded_values')
train_data_RMSE.fillna(0, inplace=True)
test_data_RMSE.fillna(0, inplace=True)

train_data_RMSE = train_data_RMSE.values
test_data_RMSE = test_data_RMSE.values

print(train_data_RMSE)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         6.0002003  0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 6.00011127 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [195]:
item_similarity = pairwise_distances(train_data_RMSE.T, metric='cosine')

In [196]:
def predict(ratings, similarity, type='item'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [197]:
item_prediction = predict(train_data_RMSE, item_similarity, type='item')
item_prediction

array([[0.14399411, 0.10318769, 0.1138149 , ..., 0.14406043, 0.14068588,
        0.1466523 ],
       [0.37490165, 0.33108886, 0.37136261, ..., 0.34572563, 0.37673911,
        0.39271643],
       [0.56628547, 0.38717466, 0.50818124, ..., 0.54807348, 0.55964305,
        0.58337723],
       ...,
       [0.35890074, 0.34114306, 0.33999822, ..., 0.18417742, 0.35065509,
        0.36552619],
       [0.59628615, 0.51266981, 0.58103221, ..., 0.55981207, 0.58765434,
        0.61257646],
       [0.10191607, 0.10556282, 0.10379754, ..., 0.07977859, 0.09957458,
        0.10379749]])

In [198]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [199]:
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_RMSE)))

Item-based CF RMSE: 3.9374823711388984
